<a href="https://colab.research.google.com/github/jsalbr/transformers-qa-pipeline-bug/blob/main/Transformers_Issue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Answering Bug Report

https://github.com/huggingface/transformers/issues/14311

Error when handling impossible results

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install transformers

In [13]:
!transformers-cli env

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
2021-11-07 13:21:49.542092: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 13:21:49.978718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 13:21:49.979652: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 13:21:52.628150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 13:21:52.628929: I tensorflo

# Error scenario

In [3]:
from transformers import pipeline

model_name = "deepset/minilm-uncased-squad2"
qa = pipeline("question-answering", model=model_name,
              tokenizer=model_name, device=0)

In [3]:
# throws exception in transformers 4.12.3
question = "How is the weather?"
context = """Washington is the capital of the United States."""
answer = qa(question, context, handle_impossible_answer=True)
answer

ValueError: ignored

# Patch 

Do not know if this is the correct solution but it seems to work
`start_` and `end_` are np.arrays.

In [9]:
patch = """
409c409
<                 min_null_score = min(min_null_score, (start_[0] * end_[0]).item())
---
>                 min_null_score = min(min_null_score, (np.max(start_[0]) * np.max(end_[0])).item())
"""

with open("qa.patch", "w") as file:
  file.write(patch)

In [8]:
!cat qa.patch

409c409
<                 min_null_score = min(min_null_score, (start_[0] * end_[0]).item())
---
>                 min_null_score = min(min_null_score, (np.max(start_[0]) * np.max(end_[0])).item())

In [10]:
!patch /usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py -i qa.patch

patching file /usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py


In [11]:
from transformers import pipeline

In [12]:
question = "How is the weather?"
context = """Washington is the capital of the United States."""
answer = qa(question, context, handle_impossible_answer=True)
answer

{'answer': '', 'end': 0, 'score': 0.9953430891036987, 'start': 0}